In [1]:
#bibliotecas 
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def calcula_resultante(f, a, b):
    fa = f(a)
    fb = f(b)

    if fa

In [2]:
#função para integração das funções distribuição pelo método dos retângulos
def integracao(funcao,a, b):
  t = np.linspace(a,b,10000)
  largura = t[1]-t[0]
  integral = np.cumsum(funcao(t[:-1]))*largura
  return integral[len(integral) - 1]

In [ ]:
#inicialização dos valores mínimos, máximos das funções
#inicialização dos valores de Xmin e Xmax 
distribuicao = np.array([[0,300],
                         [300,300],
                         [400,400]])
comprimentos_distribuicao = np.array([[0,3],
                                      [3,5],
                                      [5,10]])


In [ ]:
#determinacao das respectivas funcoes lineares
#calculo da força resultante da distribuicao integrando
#obtencao da posicao da forca resultante (x_barra)
funcoes = list()
coef = list()
Forca_resultante = 0;
Momento = 0;
for i in range(0, len(distribuicao)):
  coef.append((distribuicao[i][1] - distribuicao[i][0]) / (comprimentos_distribuicao[i][1] - comprimentos_distribuicao[i][0]))
  funcoes.append(lambda x, i = i: (coef[i]*(x - comprimentos_distribuicao[i][0]) + distribuicao[i][0]))
  Forca_resultante += integracao(funcoes[i],comprimentos_distribuicao[i][0],comprimentos_distribuicao[i][1])
  Momento += integracao(lambda x : x*funcoes[i](x),comprimentos_distribuicao[i][0],comprimentos_distribuicao[i][1])
  
x_barra = Momento / Forca_resultante
Forca_resultante = np.floor(Forca_resultante)
print(Forca_resultante)
x_barra


np.float64(2865.0)